Three functional blocks

1. Transcriptomics, Proteomics, Active Gene IDs
2. Create or load tissue specific model
3. Disease Analysis: Differential Expressions
4. Knock out simulation: Score Genes, Generate Drug List

In [ ]:
import sys
import os
import pandas
import numpy
import json
from subprocess import call
from project import configs
print(configs.rootdir)

## Step 1: Transcriptomics and Proteomics Generation

In [ ]:
transcriptomics_config_file = 'GeneExpressionDataUsed.xlsx'
proteomics_data_file = 'ni.3693-S5.xlsx'
proteomics_supplementary_data_file = 'Supplementary Data 1.xlsx'

In [ ]:
cmd = ' '.join(['python3', 'transcriptomic_gen.py', 
      '-i', '"{}"'.format(transcriptomics_config_file)])
!{cmd}

In [ ]:
cmd = ' '.join(['python3', 'proteomics_gen.py', 
      '-d', '"{}"'.format(proteomics_data_file), 
      '-s', '"{}"'.format(proteomics_supplementary_data_file)])
!{cmd}

In [ ]:
cmd = ' '.join(['python3', 'merge_xomics.py', 
      '-t', '"{}"'.format(transcriptomics_config_file), 
      '-p', '"{}"'.format(proteomics_supplementary_data_file)])
!{cmd}

## Step 1.5 Create Tissue Specific Model

In [ ]:
step1_results_file = os.path.join(configs.datadir, 'step1_results_files.json')
with open(step1_results_file) as json_file:
    tissue_gene_exp = json.load(json_file)
print(tissue_gene_exp)

In [ ]:
# (input) filename of General Model 
GeneralModelFile = 'GeneralModel.mat'
# (input) filename of Tissue Gene Expression
genefile = 'merged_Th1.csv'
# (output) filename of Tissue Specific Model
tissuefile = 'Th1_SpecificModel.mat'

In [ ]:
tissue_spec_model = {}
for key,value in tissue_gene_exp.items():
    tissuefile = '{}_SpecificModel.mat'.format(key)
    tissue_spec_model[key] = tissuefile
    tissue_gene_file = value.split('/')[-1]
    tissue_gene_folder = os.path.join(configs.datadir, key)
    os.makedirs(tissue_gene_folder, exist_ok=True)
    cmd = ' '.join(['python3', 'create_tissue_specific_model.py', 
                      '-m', '"{}"'.format(GeneralModelFile), 
                      '-g', '"{}"'.format(tissue_gene_file),
                      '-o', '"{}"'.format(tissuefile)])
    !{cmd}

print(tissue_spec_model)

## Step 2: Disease Gene Analysis
Differential Expression Analysis

Only 1 disease to be analyzed, output files in data folder

In [ ]:
disease_gene_file = 'Disease_Gene_Analyzed.xlsx'

In [ ]:
cmd = ' '.join(['python3', 'disease_analysis.py', 
              '-i', '"{}"'.format(disease_gene_file)])
!{cmd}

In [ ]:
step2_results_file = os.path.join(configs.datadir, 'step2_results_files.json')
with open(step2_results_file) as json_file:
    disease_files = json.load(json_file)
print(disease_files)

## Step 3: Knock Out Simulation
Knock out simulation and repurposing drug List generation

Instruction: Download `Repurposing_Hub_export.txt` from [Drug Repurposing Hub](https://clue.io/repurposing-app) to `/pipelines/data/`

In [ ]:
tissue_spec_model

In [ ]:
RA_Down = disease_files['DN_Reg']
RA_Up = disease_files['UP_Reg']

drug_raw_file = 'Repurposing_Hub_export.txt'

In [ ]:
for key,value in tissue_spec_model.items():
    tissueSpecificModelfile = value
    tissue_gene_folder = os.path.join(configs.datadir, key)
    os.makedirs(tissue_gene_folder, exist_ok=True)
    inhibitors_file = '{}_inhibitors_Entrez.txt'.format(key)
    cmd = ' '.join(['python3' , 'knock_out_simulation.py',
                  '-t', tissueSpecificModelfile,
                  '-i', inhibitors_file,
                  '-u', RA_Up,
                  '-d', RA_Down,
                  '-f', key,
                  '-r', drug_raw_file])
    !{cmd}
    
    # copy generated output to output folder
    cmd = ' '.join(['cp', '-a', os.path.join(configs.datadir, key), configs.outputdir])
    !{cmd}
    #break


#### To use customized model, please specify `tissue_spec_model` manually.

In [ ]:
# datafolder = 'Th1'
# tissueSpecificModelfile = 'Th1_Cell_SpecificModel4manuscript.mat'
# inhibitors_file = 'Th1_inhibitors_Entrez.txt'

# RA_Down = 'RA_DOWN.txt'
# RA_Up = 'RA_UP.txt'
# drug_raw_file = 'Repurposing_Hub_export.txt'
# # drug_pre_file = 'Repurposing_Hub_Preproc.csv'


In [ ]:
# cmd = ' '.join(['python3' , 'knock_out_simulation.py',
#               '-t', tissueSpecificModelfile,
#               '-i', inhibitors_file,
#               '-u', RA_Up,
#               '-d', RA_Down,
#               '-f', datafolder,
#               '-r', drug_raw_file])